In [20]:
!pip install pandas requests openpyxl


In [21]:
!pip install googlemaps

In [84]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import googlemaps
import json


In [38]:
fire_data_path = r"/fire_incidents.csv"

fire_station_data_path = r"/fire-station-locations.csv"


# Read the CSV file into a DataFrame
df_fire_data = pd.read_csv(fire_data_path)
df_fire_station_data = pd.read_csv(fire_station_data_path)

# Display the first few rows of the DataFrame
#df_fire_station_data.head()


<ipython-input-38-15fbe51a4ddf>:7: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fire_data = pd.read_csv(fire_data_path)


In [39]:
df_fire_data.isna().sum()

,0
_id,0
Area_of_Origin,9002
Building_Status,16062
Business_Impact,16066
Civilian_Casualties,7133
Count_of_Persons_Rescued,223
Estimated_Dollar_Loss,8979
Estimated_Number_Of_Persons_Displaced,16065
Exposures,25418
Ext_agent_app_or_defer_time,9012


In [40]:
# dropping the null rows for lat long and tfs arrival time

df_fire_data.dropna(subset=['Latitude', 'Longitude', 'TFS_Arrival_Time','Incident_Station_Area'], inplace=True)

In [41]:
df_fire_data.isna().sum()

,0
_id,0
Area_of_Origin,9002
Building_Status,16060
Business_Impact,16064
Civilian_Casualties,7133
Count_of_Persons_Rescued,222
Estimated_Dollar_Loss,8979
Estimated_Number_Of_Persons_Displaced,16063
Exposures,25416
Ext_agent_app_or_defer_time,9011


In [60]:
# Convert 'Incident_Station_Area' to numeric, ignoring errors
df_fire_data['Incident_Station_Area'] = pd.to_numeric(df_fire_data['Incident_Station_Area'], errors='coerce')

# Fill NaN values with a suitable replacement (e.g., 0 or the mean/median)
df_fire_data['Incident_Station_Area'].fillna(0, inplace=True)

# Convert the column to integers (values will be truncated if they have decimal part after filling NaN values)
df_fire_data['Incident_Station_Area'] = df_fire_data['Incident_Station_Area'].astype(int)

<ipython-input-60-8c720d8be6ee>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_fire_data['Incident_Station_Area'].fillna(0, inplace=True)


In [67]:
set1 = df_fire_station_data['STATION'].unique()
set1

array([211,   0, 342, 214, 426, 121, 412, 134, 331, 115, 132, 433, 341,
       311, 323, 234, 221, 224, 244, 123, 131, 223, 231, 312, 243, 444,
       423, 212, 227, 222, 213, 445, 413, 116, 432, 225, 431, 415, 113,
       133, 421, 142, 344, 315, 325, 443, 146, 143, 245, 333, 241, 145,
       235, 442, 411, 112, 111, 114, 334, 324, 313, 335, 345, 135, 122,
       242, 125, 435, 232, 321, 233, 141, 434, 326, 425, 422, 346, 226,
       343, 215, 314, 332, 144, 441, 322])

In [68]:
set2 = df_fire_data['Incident_Station_Area'].unique()
set2

array([441, 116, 221, 133, 132, 215, 235, 231, 332, 426, 225, 325, 226,
       341, 421, 244, 141, 115, 415, 431, 331, 413, 314, 333, 311, 145,
       143, 342, 443, 312, 223, 134, 214, 434, 423, 233, 114, 112, 224,
       326, 212, 343, 135, 125, 315, 234, 324, 113, 142, 146, 313, 442,
       222, 241, 345, 232, 121, 432, 425, 334, 411, 445, 243, 323, 435,
       213, 422, 412, 123, 344, 111, 242, 321, 433, 245, 444, 211, 131,
       322, 122, 335, 227, 346, 144,   0])

In [69]:
# find values that are in set2 that are not in set1

result = set(set2) - set(set1)
result

set()

In [64]:
df_fire_data['Incident_Station_Area'].isna().sum()

0

In [65]:
# Find all rows where 'Incident_Station_Area' is 424
station_424_rows = df_fire_data[df_fire_data['Incident_Station_Area'] == 424]

station_424_rows

,_id,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties
7711,3700707,64 - Porch or Balcony,01 - Normal (no change),1 - No business interruption,0.0,0.0,2500.0,0.0,NaN,2011-07-13T02:33:37,...,4 - Remote from fire – smoke did not reach alarm,4 - Interconnected,7 - Not applicable: Occupant(s) first alerted ...,2 - Confined to part of room/area of origin,2 - Did not activate: remote from fire,2 - Partial sprinkler system present,2 - Fire with no evidence from street,2011-07-13T02:21:14,2011-07-13T02:26:42,0.0
7724,3700720,97 - Other - unclassified,NaN,NaN,0.0,0.0,1000.0,NaN,NaN,2011-08-19T14:17:00,...,NaN,NaN,NaN,NaN,NaN,NaN,3 - Fire with smoke showing only - including v...,2011-08-19T14:11:59,2011-08-19T14:16:54,0.0
8210,3701206,97 - Other - unclassified,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,300.0,0.0,NaN,2012-02-02T21:25:00,...,4 - Remote from fire – smoke did not reach alarm,1 - Battery operated,7 - Not applicable: Occupant(s) first alerted ...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,1 - Fire extinguished prior to arrival,2012-02-02T21:19:23,2012-02-02T21:24:13,0.0
8242,3701238,78 - Attached Deck,01 - Normal (no change),1 - No business interruption,0.0,0.0,50000.0,997.0,NaN,2013-08-20T03:14:55,...,98 - Not applicable: Alarm operated OR presenc...,1 - Battery operated,9 - Undetermined,8 - Entire Structure,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,2 - Fire with no evidence from street,2013-08-20T03:06:24,2013-08-20T03:11:53,0.0
8580,3701576,79 - Other Outside Area,01 - Normal (no change),1 - No business interruption,0.0,0.0,200.0,0.0,NaN,2012-07-03T00:45:00,...,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,7 - Not applicable: Occupant(s) first alerted ...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,1 - Fire extinguished prior to arrival,2012-07-03T00:41:19,2012-07-03T00:44:55,0.0
9086,3702082,49 - Other Storage Area,NaN,NaN,0.0,0.0,300.0,NaN,NaN,2012-02-26T17:55:00,...,NaN,NaN,NaN,NaN,NaN,NaN,1 - Fire extinguished prior to arrival,2012-02-26T17:50:17,2012-02-26T17:54:59,0.0
9143,3702139,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),1.0,0.0,1500.0,2.0,NaN,2012-09-07T07:56:11,...,98 - Not applicable: Alarm operated OR presenc...,1 - Battery operated,1 - All persons (at risk of injury) self evacu...,"7 - Spread to other floors, confined to building",8 - Not applicable - no sprinkler system present,3 - No sprinkler system,1 - Fire extinguished prior to arrival,2012-09-07T07:51:28,2012-09-07T07:54:43,0.0
9402,3702398,42 - Garage,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,40000.0,0.0,1.0,2013-12-16T04:36:39,...,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,"8 - Not applicable: No alarm, no persons present",10 - Spread beyond building of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,8 - Exposure involved,2013-12-16T04:29:01,2013-12-16T04:35:03,0.0
10601,3703597,64 - Porch or Balcony,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,50.0,0.0,NaN,2011-06-11T16:43:26,...,4 - Remote from fire – smoke did not reach alarm,1 - Battery operated,7 - Not applicable: Occupant(s) first alerted ...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 

In [66]:


# Drop rows where 'Incident_Station_Area' is 424
df_fire_data = df_fire_data[df_fire_data['Incident_Station_Area'] != 424]

In [70]:
# Removing all rows where the fire station number is 0 since those locations have closed

df_fire_station_data = df_fire_station_data[df_fire_station_data['STATION'] != 0]

In [72]:
# Remove rows where 'Incident_Station_Area' is 0
df_fire_data = df_fire_data[df_fire_data['Incident_Station_Area'] != 0]

In [73]:
df_fire_data['Incident_Station_Area'].value_counts()

,count
Incident_Station_Area,
313,1087
314,974
426,933
325,897
332,804
...,...
131,157
122,157
111,155


Feature Engineering the Response Time


In [75]:
df_fire_data['TFS_Alarm_Time'] = pd.to_datetime(df_fire_data['TFS_Alarm_Time'])
df_fire_data['TFS_Arrival_Time'] = pd.to_datetime(df_fire_data['TFS_Arrival_Time'])

# Calculate the response time in seconds
df_fire_data['Response_Time_Seconds'] = (df_fire_data['TFS_Arrival_Time'] - df_fire_data['TFS_Alarm_Time']).dt.total_seconds()

In [76]:
df_fire_data['Response_Time_Seconds'].head(20)

,Response_Time_Seconds
0,342.0
1,288.0
2,410.0
3,268.0
4,336.0
5,238.0
6,352.0
7,421.0
8,210.0
9,329.0


In [79]:
# Merge the two dataframes
df_merged = pd.merge(df_fire_data, df_fire_station_data, left_on='Incident_Station_Area', right_on='STATION', how='inner')

# Display the merged dataframe
df_merged

,_id_x,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,ID,STATION,YEAR_BUILD,WARD,WARD_NAME,TYPE_DESC,PUBLIC_ED_OFFICE,FIRE_PREV_OFFICE,FIRE_OTHER,geometry
0,3692996,81 - Engine Area,NaN,NaN,0.0,0.0,15000.0,NaN,NaN,2018-02-24T21:12:00,...,6,441,1964,1,Etobicoke North,Fire Station,NaN,NaN,West Training Centre / Fire Training Tower,"{'type': 'MultiPoint', 'coordinates': [[-79.57..."
1,3692997,"75 - Trash, rubbish area (outside)",NaN,NaN,0.0,0.0,50.0,NaN,NaN,2018-02-24T21:29:42,...,1,116,2007,17,Don Valley North,Fire Station,NaN,"Fire Prevention Office (E, N)",Division Commander / Platoon Chiefs,"{'type': 'MultiPoint', 'coordinates': [[-79.36..."
2,3692998,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,95,221,2014,20,Scarborough Southwest,Fire Station,NaN,NaN,Division Commander / Platoon Chiefs,"{'type': 'MultiPoint', 'coordinates': [[-79.25..."
3,3692999,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,NaN,2018-02-25T14:19:25,...,94,133,1962,5,York South-Weston,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.48..."
4,3693000,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,16,132,1999,8,Eglinton-Lawrence,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.42..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32897,3725920,22 - Sleeping Area or Bedroom (inc. patients r...,01 - Normal (no change),1 - No business interruption,NaN,0.0,0.0,0.0,0.0,2023-01-01T07:29:02,...,80,114,1989,18,Willowdale,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.41..."
32898,3725921,64 - Porch or Balcony,01 - Normal (no change),8 - Not applicable (not a business),NaN,0.0,3000.0,0.0,0.0,2023-01-01T10:57:07,...,45,322,1994,14,Toronto-Danforth,Fire Station,NaN,NaN,Accreditation,"{'type': 'MultiPoint', 'coordinates': [[-79.34..."
32899,3725922,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),NaN,0.0,1000.0,0.0,0.0,2023-01-01T12:28:05,...,72,213,1974,23,Scarborough North,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.22..."
32900,3725923,99 - Undetermined (formerly 98),NaN,NaN,NaN,0.0,5000.0,NaN,0.0,2023-01-01T13:17:38,...,64,244,1971,22,Scarborough-Agincourt,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.29..."


In [80]:
#combining the address

df_merged['Combined_Address'] = df_merged['ADDRESS'] + ', ' + df_merged['MUNICIPALITY_NAME']

In [81]:
# Display the final dataframe
df_merged.head()

,_id_x,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,STATION,YEAR_BUILD,WARD,WARD_NAME,TYPE_DESC,PUBLIC_ED_OFFICE,FIRE_PREV_OFFICE,FIRE_OTHER,geometry,Combined_Address
0,3692996,81 - Engine Area,NaN,NaN,0.0,0.0,15000.0,NaN,NaN,2018-02-24T21:12:00,...,441,1964,1,Etobicoke North,Fire Station,NaN,NaN,West Training Centre / Fire Training Tower,"{'type': 'MultiPoint', 'coordinates': [[-79.57...","947 Martin Grove Rd, Etobicoke"
1,3692997,"75 - Trash, rubbish area (outside)",NaN,NaN,0.0,0.0,50.0,NaN,NaN,2018-02-24T21:29:42,...,116,2007,17,Don Valley North,Fire Station,NaN,"Fire Prevention Office (E, N)",Division Commander / Platoon Chiefs,"{'type': 'MultiPoint', 'coordinates': [[-79.36...","255 Esther Shiner Blvd, North York"
2,3692998,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,221,2014,20,Scarborough Southwest,Fire Station,NaN,NaN,Division Commander / Platoon Chiefs,"{'type': 'MultiPoint', 'coordinates': [[-79.25...","2575 Eglinton Ave E, Scarborough"
3,3692999,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,NaN,2018-02-25T14:19:25,...,133,1962,5,York South-Weston,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.48...","1505 Lawrence Ave W, North York"
4,3693000,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,132,1999,8,Eglinton-Lawrence,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.42...","476 Lawrence Ave W, North York"


In [82]:
df_merged['geometry']

,geometry
0,"{'type': 'MultiPoint', 'coordinates': [[-79.57..."
1,"{'type': 'MultiPoint', 'coordinates': [[-79.36..."
2,"{'type': 'MultiPoint', 'coordinates': [[-79.25..."
3,"{'type': 'MultiPoint', 'coordinates': [[-79.48..."
4,"{'type': 'MultiPoint', 'coordinates': [[-79.42..."
...,...
32897,"{'type': 'MultiPoint', 'coordinates': [[-79.41..."
32898,"{'type': 'MultiPoint', 'coordinates': [[-79.34..."
32899,"{'type': 'MultiPoint', 'coordinates': [[-79.22..."
32900,"{'type': 'MultiPoint', 'coordinates': [[-79.29..."


In [86]:
# Parse longitude and latitude directly using lambda functions
df_merged["station_longitude"] = df_merged["geometry"].apply(lambda loc: json.loads(loc.replace("'", "\""))["coordinates"][0][0])
df_merged["station_latitude"] = df_merged["geometry"].apply(lambda loc: json.loads(loc.replace("'", "\""))["coordinates"][0][1])

In [88]:
df_merged['station_latitude']

,station_latitude
0,43.694960
1,43.769146
2,43.734790
3,43.707185
4,43.719798
...,...
32897,43.772636
32898,43.690986
32899,43.795407
32900,43.781682


In [89]:
df_merged['station_longitude']

,station_longitude
0,-79.571920
1,-79.365064
2,-79.255063
3,-79.482185
4,-79.428572
...,...
32897,-79.415355
32898,-79.342344
32899,-79.225962
32900,-79.299115


In [90]:
gmaps = googlemaps.Client(key='')

def calculate_distance_time(row):
    try:
        result = gmaps.distance_matrix(
            origins=(row['station_latitude'], row['station_longitude']),
            destinations=(row['Latitude'], row['Longitude']),
            mode="driving",
            units="metric"
        )
        distance = result['rows'][0]['elements'][0]['distance']['value']
        duration = result['rows'][0]['elements'][0]['duration']['value']

        return distance, duration

    except Exception as e:
        print(f"Error calculating distance/time: {e} for row: {row}")
        return None, None


df_merged[['distance_from_nearest_station', 'time_from_nearest_station']] = df_merged.apply(calculate_distance_time, axis=1, result_type='expand')

print(df_merged[['distance_from_nearest_station', 'time_from_nearest_station']])

Error calculating distance/time: 'distance' for row: _id_x                                                            3723676
Area_of_Origin                                                       NaN
Building_Status                                                      NaN
Business_Impact                                                      NaN
Civilian_Casualties                                                  NaN
                                             ...                        
FIRE_OTHER                                                           NaN
geometry               {'type': 'MultiPoint', 'coordinates': [[-79.41...
Combined_Address                            12 Canterbury Pl, North York
station_longitude                                             -79.415355
station_latitude                                               43.772636
Name: 30653, Length: 65, dtype: object
       distance_from_nearest_station  time_from_nearest_station
0                             6297.0     

In [91]:
df_merged.head()

,_id_x,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,TYPE_DESC,PUBLIC_ED_OFFICE,FIRE_PREV_OFFICE,FIRE_OTHER,geometry,Combined_Address,station_longitude,station_latitude,distance_from_nearest_station,time_from_nearest_station
0,3692996,81 - Engine Area,NaN,NaN,0.0,0.0,15000.0,NaN,NaN,2018-02-24T21:12:00,...,Fire Station,NaN,NaN,West Training Centre / Fire Training Tower,"{'type': 'MultiPoint', 'coordinates': [[-79.57...","947 Martin Grove Rd, Etobicoke",-79.571920,43.694960,6297.0,507.0
1,3692997,"75 - Trash, rubbish area (outside)",NaN,NaN,0.0,0.0,50.0,NaN,NaN,2018-02-24T21:29:42,...,Fire Station,NaN,"Fire Prevention Office (E, N)",Division Commander / Platoon Chiefs,"{'type': 'MultiPoint', 'coordinates': [[-79.36...","255 Esther Shiner Blvd, North York",-79.365064,43.769146,2721.0,283.0
2,3692998,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,Fire Station,NaN,NaN,Division Commander / Platoon Chiefs,"{'type': 'MultiPoint', 'coordinates': [[-79.25...","2575 Eglinton Ave E, Scarborough",-79.255063,43.734790,1579.0,244.0
3,3692999,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,NaN,2018-02-25T14:19:25,...,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.48...","1505 Lawrence Ave W, North York",-79.482185,43.707185,382.0,83.0
4,3693000,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.42...","476 Lawrence Ave W, North York",-79.428572,43.719798,1626.0,234.0


In [93]:
# Calculate the response delay
df_merged['response_delay'] = df_merged['Response_Time_Seconds'] - df_merged['time_from_nearest_station']

In [94]:
# Save the DataFrame to a CSV file
df_merged.to_csv('/Google_Maps_Api_distance_output.csv', index=False)

In [98]:
# Calculate the average delay by incident station area
average_delay_by_station = df_merged.groupby('Incident_Station_Area')['response_delay'].mean().reset_index()

# Rename the columns for clarity
average_delay_by_station.columns = ['Incident_Station_Area', 'Average_Response_Delay']

# Display the new DataFrame
average_delay_by_station

,Incident_Station_Area,Average_Response_Delay
0,111,78.225806
1,112,66.051282
2,113,98.602564
3,114,67.318000
4,115,85.870588
...,...,...
79,441,-0.615071
80,442,52.524046
81,443,12.265244
82,444,154.974576


In [100]:
# Save the average delay by station to a CSV file
average_delay_by_station.to_csv('/average_delay_by_station.csv', index=False)